In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gzip
import json

import pandas as pd

from matpowercaseframes import CaseFrames

In [3]:
pd.set_option("display.max_columns", None)

## UnitCommitment.jl Data

In [4]:
CASE_NAME = "../data/case14/2017-01-01.json.gz"
with gzip.open(CASE_NAME, "rt", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
# TODO: convert `UnitCommitment.jl` format to matpowercaseframes format
data.keys()

dict_keys(['SOURCE', 'Parameters', 'Generators', 'Transmission lines', 'Contingencies', 'Buses', 'Reserves'])

## MATPOWER Data

In [6]:
CASE_NAME = "case14.m"
cf_14 = CaseFrames(CASE_NAME)

In [ ]:
import numpy as np
import pandas as pd


# Helper functions for data processing
def _extract_scalar_bounds(points):
    """Extract min/max from possibly nested array structure for time-varying limits."""
    if not isinstance(points, list):
        return points, points
    if len(points) == 0:
        return 0.0, 0.0
    # Handle nested arrays like [5.0, [10.0, 12.0, 15.0, 20.0]]
    flat = []
    for p in points:
        if isinstance(p, list):
            flat.extend(p)
        else:
            flat.append(p)
    return min(flat), max(flat)


def _get_last_value(val):
    """Get last value from time-series or return scalar."""
    if isinstance(val, list) and len(val) > 0:
        return val[-1]
    return val


def _is_time_series(val):
    """Check if value is a time series (list with multiple values)."""
    return isinstance(val, list) and len(val) > 0


def _extract_parameters(parameters: dict) -> dict:
    """Extract and set default values for system parameters."""
    version = parameters.get("Version", "0.0")
    return {
        "version": f"UnitCommitment.jl-{version}",
        "baseMVA": 100.0,  # Standard MATPOWER base
        "time_horizon_min": parameters.get("Time horizon (min)"),
        "time_horizon_h": parameters.get("Time horizon (h)"),
        "time_step_min": parameters.get("Time step (min)", 60),
        "power_balance_penalty": parameters.get("Power balance penalty ($/MW)", 1000.0),
        "scenario_name": parameters.get("Scenario name", "s1"),
        "scenario_weight": parameters.get("Scenario weight", 1.0),
    }


def _infer_generator_type(gen_data: dict) -> str:
    """
    Infer generator type from data.

    v0.4+: Use explicit "Type" field
    v0.3: Infer from presence of "Production cost curve (MW)"
    """
    if "Type" in gen_data:
        return gen_data["Type"]
    # Version 0.3: Infer type
    if "Production cost curve (MW)" in gen_data:
        return "Thermal"
    return "Profiled"


def _reorder_columns(df: pd.DataFrame, standard_cols: list) -> pd.DataFrame:
    """Reorder DataFrame columns: standard MATPOWER columns first, then extras."""
    if df.empty:
        return df

    # Get columns that exist in both df and standard_cols (in standard order)
    existing_standard = [col for col in standard_cols if col in df.columns]

    # Get extra columns not in standard_cols
    extra_cols = [col for col in df.columns if col not in standard_cols]

    # Reorder: standard first, then extras
    return df[existing_standard + extra_cols]


def _ensure_matpower_gen_columns(gen_df: pd.DataFrame) -> pd.DataFrame:
    """Ensure all required MATPOWER gen columns exist with proper defaults."""
    from matpowercaseframes.constants import COLUMNS

    standard_cols = COLUMNS["gen"]
    defaults = {
        "PC1": 0.0,
        "PC2": 0.0,
        "QC1MIN": 0.0,
        "QC1MAX": 0.0,
        "QC2MIN": 0.0,
        "QC2MAX": 0.0,
        "RAMP_Q": 0.0,
        "APF": 0.0,
        "MU_PMAX": 0.0,
        "MU_PMIN": 0.0,
        "MU_QMAX": 0.0,
        "MU_QMIN": 0.0,
    }

    for col in standard_cols:
        if col not in gen_df.columns:
            gen_df[col] = defaults.get(col, 0.0)

    return _reorder_columns(gen_df, standard_cols)


def _ensure_matpower_bus_columns(bus_df: pd.DataFrame) -> pd.DataFrame:
    """Ensure all required MATPOWER bus columns exist with proper defaults."""
    from matpowercaseframes.constants import COLUMNS

    standard_cols = COLUMNS["bus"]
    defaults = {
        "LAM_P": 0.0,
        "LAM_Q": 0.0,
        "MU_VMAX": 0.0,
        "MU_VMIN": 0.0,
    }

    for col in standard_cols:
        if col not in bus_df.columns:
            bus_df[col] = defaults.get(col, 0.0)

    return _reorder_columns(bus_df, standard_cols)


def _ensure_matpower_branch_columns(branch_df: pd.DataFrame) -> pd.DataFrame:
    """Ensure all required MATPOWER branch columns exist with proper defaults."""
    from matpowercaseframes.constants import COLUMNS

    standard_cols = COLUMNS["branch"]
    defaults = {
        "PF": 0.0,
        "QF": 0.0,
        "PT": 0.0,
        "QT": 0.0,
        "MU_SF": 0.0,
        "MU_ST": 0.0,
        "MU_ANGMIN": 0.0,
        "MU_ANGMAX": 0.0,
    }

    for col in standard_cols:
        if col not in branch_df.columns:
            branch_df[col] = defaults.get(col, 0.0)

    return _reorder_columns(branch_df, standard_cols)


def _ensure_matpower_gencost_columns(gencost_df: pd.DataFrame) -> pd.DataFrame:
    """Ensure gencost columns follow MATPOWER order, then extras."""
    from matpowercaseframes.constants import COLUMNS

    standard_cols = COLUMNS["gencost"]
    return _reorder_columns(gencost_df, standard_cols)


def _process_profiled_generator(gen_name: str, gen_data: dict, gen_bus: str) -> tuple:
    """Process a profiled (renewable/hydro) generator."""
    cost_per_mw = gen_data["Cost ($/MW)"]  # Required
    min_power = gen_data.get("Minimum power (MW)", 0.0)  # Default: 0.0
    max_power = gen_data["Maximum power (MW)"]  # Required

    # Standard MATPOWER columns first
    matpower_gen = {
        "GEN_BUS": gen_bus,
        "PG": 0.0,
        "QG": 0.0,
        "QMAX": 0.0,
        "QMIN": 0.0,
        "VG": 1.0,
        "MBASE": 100.0,
        "GEN_STATUS": 1,
        "PMAX": max_power,
        "PMIN": min_power,
        # UnitCommitment.jl extras (after standard columns)
        "GEN_NAME": gen_name,
        "TYPE": "Profiled",
        "COST_PER_MW": cost_per_mw,
        "MIN_POWER": min_power,
        "MAX_POWER": max_power,
    }

    matpower_gencost = {
        "MODEL": 1,
        "STARTUP": 0.0,
        "SHUTDOWN": 0.0,
        "NCOST": 2,
        "X1": min_power,
        "Y1": min_power * cost_per_mw,
        "X2": max_power,
        "Y2": max_power * cost_per_mw,
    }

    return matpower_gen, matpower_gencost


def _process_thermal_generator(gen_name: str, gen_data: dict, gen_bus: str) -> tuple:
    """Process a thermal (coal/gas/nuclear) generator."""
    # Required fields
    mw_points = gen_data["Production cost curve (MW)"]
    cost_points = gen_data["Production cost curve ($)"]
    initial_status = gen_data["Initial status (h)"]
    initial_power = gen_data["Initial power (MW)"]

    pmin, pmax = _extract_scalar_bounds(mw_points)
    gen_status = 1 if initial_status > 0 else 0

    # Ramp limits with defaults
    ramp_up = gen_data.get("Ramp up limit (MW)", float("inf"))
    ramp_down = gen_data.get("Ramp down limit (MW)", float("inf"))

    # Standard MATPOWER columns first
    matpower_gen = {
        "GEN_BUS": gen_bus,
        "PG": initial_power,
        "QG": 0.0,
        "QMAX": 0.0,
        "QMIN": 0.0,
        "VG": 1.0,
        "MBASE": 100.0,
        "GEN_STATUS": gen_status,
        "PMAX": pmax,
        "PMIN": pmin,
    }

    # Compute MATPOWER ramp rates if available
    if ramp_up != float("inf"):
        matpower_gen["RAMP_AGC"] = ramp_up / 60.0
        matpower_gen["RAMP_10"] = ramp_up / 6.0
        matpower_gen["RAMP_30"] = ramp_up / 2.0

    # UnitCommitment.jl extras (after standard columns)
    matpower_gen.update(
        {
            "GEN_NAME": gen_name,
            "TYPE": "Thermal",
            "RAMP_UP": ramp_up,
            "RAMP_DOWN": ramp_down,
            "RAMP_UP_SU": gen_data.get("Startup limit (MW)", float("inf")),
            "RAMP_DOWN_SD": gen_data.get("Shutdown limit (MW)", float("inf")),
            "MUT": gen_data.get("Minimum uptime (h)", 1),
            "MDT": gen_data.get("Minimum downtime (h)", 1),
            "INITIAL_STATUS": initial_status,
            "INITIAL_POWER": initial_power,
            "MUST_RUN": gen_data.get("Must run?", False),
            "RESERVE_ELIGIBILITY": gen_data.get("Reserve eligibility", []),
            "COMMITMENT_STATUS": gen_data.get("Commitment status"),
        }
    )

    # Gencost with startup costs
    startup_costs = gen_data.get("Startup costs ($)", [0.0])
    startup_delays = gen_data.get("Startup delays (h)", [1])

    matpower_gencost = {
        "MODEL": 1,  # Piecewise linear only
        "STARTUP": startup_costs[0] if len(startup_costs) > 0 else 0.0,
        "SHUTDOWN": 0.0,
        "NCOST": len(mw_points) if isinstance(mw_points, list) else 1,
    }

    # Production cost curve points (X1, Y1, X2, Y2, ...)
    if isinstance(mw_points, list):
        for i, (mw, cost) in enumerate(zip(mw_points, cost_points), start=1):
            matpower_gencost[f"X{i}"] = mw
            matpower_gencost[f"Y{i}"] = cost

    # Store all startup cost tiers as extra columns
    for i, (delay, cost) in enumerate(zip(startup_delays, startup_costs), start=1):
        matpower_gencost[f"STARTUP_DELAY_{i}"] = delay
        matpower_gencost[f"STARTUP_COST_{i}"] = cost

    return matpower_gen, matpower_gencost


def _process_generators(data: dict) -> tuple:
    """Process all generators and return gen and gencost DataFrames."""
    gen_list = []
    gencost_list = []

    for gen_name, gen_data in data.get("Generators", {}).items():
        gen_type = _infer_generator_type(gen_data)
        gen_bus = gen_data["Bus"]  # Required

        if gen_type == "Profiled":
            gen, gencost = _process_profiled_generator(gen_name, gen_data, gen_bus)
        else:  # Thermal
            gen, gencost = _process_thermal_generator(gen_name, gen_data, gen_bus)

        gen_list.append(gen)
        gencost_list.append(gencost)

    # Create DataFrames
    gen_df = pd.DataFrame(gen_list) if gen_list else pd.DataFrame()
    gencost_df = pd.DataFrame(gencost_list) if gencost_list else pd.DataFrame()

    # Ensure MATPOWER compatibility and column order
    gen_df = _ensure_matpower_gen_columns(gen_df)
    gencost_df = _ensure_matpower_gencost_columns(gencost_df)

    return gen_df, gencost_df


def _process_buses(data: dict) -> tuple:
    """Process all buses and return bus and bus_ts DataFrames."""
    bus_list = []
    bus_ts_list = []

    for bus_name, bus_data in data.get("Buses", {}).items():
        load_mw = bus_data["Load (MW)"]  # Required

        # Check if load is time-series
        if _is_time_series(load_mw):
            # Store first value in bus, time series in bus_ts (long format)
            first_load = load_mw[0]
            for time_idx, load_val in enumerate(load_mw):
                # PD column
                bus_ts_list.append(
                    {
                        "BUS_NAME": bus_name,
                        "TIME": time_idx,
                        "COLUMN": "PD",
                        "VALUE": load_val,
                    }
                )
                # QD column (always 0.0 for UnitCommitment.jl)
                bus_ts_list.append(
                    {
                        "BUS_NAME": bus_name,
                        "TIME": time_idx,
                        "COLUMN": "QD",
                        "VALUE": 0.0,
                    }
                )
        else:
            # Scalar load
            first_load = load_mw

        # Standard MATPOWER columns first
        matpower_bus = {
            "BUS_I": bus_name,
            "BUS_TYPE": 1,
            "PD": first_load,
            "QD": 0.0,
            "GS": 0.0,
            "BS": 0.0,
            "BUS_AREA": 1,
            "VM": 1.0,
            "VA": 0.0,
            "BASE_KV": 1.0,
            "ZONE": 1,
            "VMAX": 1.1,
            "VMIN": 0.9,
            # UnitCommitment.jl extras
            "BUS_NAME": bus_name,
        }
        bus_list.append(matpower_bus)

    # Create DataFrames
    bus_df = pd.DataFrame(bus_list) if bus_list else pd.DataFrame()
    bus_ts_df = pd.DataFrame(bus_ts_list) if bus_ts_list else pd.DataFrame()

    # Ensure MATPOWER compatibility and column order
    bus_df = _ensure_matpower_bus_columns(bus_df)

    return bus_df, bus_ts_df


def _process_storage(data: dict) -> pd.DataFrame:
    """Process all storage units and return DataFrame."""
    storage_list = []

    for storage_name, storage_data in data.get("Storage units", {}).items():
        min_level = storage_data.get("Minimum level (MWh)", 0.0)
        max_level = storage_data["Maximum level (MWh)"]  # Required

        matpower_storage = {
            "STORAGE_NAME": storage_name,
            "STORAGE_BUS": storage_data["Bus"],  # Required
            "MIN_LEVEL_MWH": min_level,
            "MAX_LEVEL_MWH": max_level,
            "ALLOW_SIMULTANEOUS": storage_data.get(
                "Allow simultaneous charging and discharging", True
            ),
            "CHARGE_COST": storage_data["Charge cost ($/MW)"],  # Required
            "DISCHARGE_COST": storage_data["Discharge cost ($/MW)"],  # Required
            "CHARGE_EFF": storage_data.get("Charge efficiency", 1.0),
            "DISCHARGE_EFF": storage_data.get("Discharge efficiency", 1.0),
            "LOSS_FACTOR": storage_data.get("Loss factor", 0.0),
            "MIN_CHARGE_RATE": storage_data.get("Minimum charge rate (MW)", 0.0),
            "MAX_CHARGE_RATE": storage_data["Maximum charge rate (MW)"],  # Required
            "MIN_DISCHARGE_RATE": storage_data.get("Minimum discharge rate (MW)", 0.0),
            "MAX_DISCHARGE_RATE": storage_data[
                "Maximum discharge rate (MW)"
            ],  # Required
            "INIT_LEVEL": storage_data.get("Initial level (MWh)", 0.0),
            "LAST_MIN_LEVEL": storage_data.get(
                "Last period minimum level (MWh)", _get_last_value(min_level)
            ),
            "LAST_MAX_LEVEL": storage_data.get(
                "Last period maximum level (MWh)", _get_last_value(max_level)
            ),
        }
        storage_list.append(matpower_storage)

    return pd.DataFrame(storage_list) if storage_list else pd.DataFrame()


def _process_price_sensitive_loads(data: dict) -> pd.DataFrame:
    """Process all price-sensitive loads and return DataFrame."""
    ps_load_list = []

    for ps_load_name, ps_load_data in data.get("Price-sensitive loads", {}).items():
        matpower_ps_load = {
            "PS_LOAD_NAME": ps_load_name,
            "PS_LOAD_BUS": ps_load_data["Bus"],  # Required
            "REVENUE": ps_load_data["Revenue ($/MW)"],  # Required
            "DEMAND": ps_load_data["Demand (MW)"],  # Required
        }
        ps_load_list.append(matpower_ps_load)

    return pd.DataFrame(ps_load_list) if ps_load_list else pd.DataFrame()


def _process_transmission_lines(data: dict) -> pd.DataFrame:
    """Process all transmission lines and return DataFrame."""
    branch_list = []

    for line_name, line_data in data.get("Transmission lines", {}).items():
        susceptance = line_data["Susceptance (S)"]  # Required

        # Standard MATPOWER columns first
        matpower_branch = {
            "F_BUS": line_data["Source bus"],  # Required
            "T_BUS": line_data["Target bus"],  # Required
            "BR_R": 0.0,
            "BR_X": 1.0 / susceptance if susceptance != 0 else 0.0,
            "BR_B": susceptance,
            "RATE_A": line_data.get("Normal flow limit (MW)", 0.0),
            "RATE_B": line_data.get("Emergency flow limit (MW)", 0.0),
            "RATE_C": line_data.get("Emergency flow limit (MW)", 0.0),
            "TAP": 0.0,
            "SHIFT": 0.0,
            "BR_STATUS": 1,
            "ANGMIN": -360.0,
            "ANGMAX": 360.0,
            # UnitCommitment.jl extras
            "BRANCH_NAME": line_name,
            "FLOW_LIMIT_PENALTY": line_data.get("Flow limit penalty ($/MW)", 5000.0),
        }
        branch_list.append(matpower_branch)

    # Create DataFrame
    branch_df = pd.DataFrame(branch_list) if branch_list else pd.DataFrame()

    # Ensure MATPOWER compatibility and column order
    branch_df = _ensure_matpower_branch_columns(branch_df)

    return branch_df


def _process_reserves(data: dict) -> tuple:
    """Process all reserves and return reserve and reserve_ts DataFrames."""
    reserve_list = []
    reserve_ts_list = []

    for reserve_name, reserve_data in data.get("Reserves", {}).items():
        amount_mw = reserve_data["Amount (MW)"]  # Required
        reserve_type = reserve_data["Type"]  # Required

        # Check if amount is time-series
        if _is_time_series(amount_mw):
            # Store first value in reserve, time series in reserve_ts (long format)
            first_amount = amount_mw[0]
            for time_idx, amount_val in enumerate(amount_mw):
                reserve_ts_list.append(
                    {
                        "RESERVE_NAME": reserve_name,
                        "TIME": time_idx,
                        "COLUMN": "AMOUNT_MW",
                        "VALUE": amount_val,
                    }
                )
        else:
            # Scalar amount
            first_amount = amount_mw

        matpower_reserve = {
            "RESERVE_NAME": reserve_name,
            "TYPE": reserve_type,
            "AMOUNT_MW": first_amount,
            "SHORTFALL_PENALTY": reserve_data.get("Shortfall penalty ($/MW)", -1),
        }
        reserve_list.append(matpower_reserve)

    reserve_df = pd.DataFrame(reserve_list) if reserve_list else pd.DataFrame()
    reserve_ts_df = pd.DataFrame(reserve_ts_list) if reserve_ts_list else pd.DataFrame()

    return reserve_df, reserve_ts_df


def _process_contingencies(data: dict) -> pd.DataFrame:
    """Process all contingencies and return DataFrame."""
    contingency_list = []

    for cont_name, cont_data in data.get("Contingencies", {}).items():
        matpower_contingency = {
            "CONTINGENCY_NAME": cont_name,
            "AFFECTED_GENERATORS": cont_data.get("Affected generators", []),
            "AFFECTED_LINES": cont_data.get("Affected lines", []),
        }
        contingency_list.append(matpower_contingency)

    return pd.DataFrame(contingency_list) if contingency_list else pd.DataFrame()


def unitcommitmentjl_to_matpower(data: dict):
    """
    Convert UnitCommitment.jl JSON format to MATPOWER CaseFrames format.

    Args:
        data: Dictionary loaded from UnitCommitment.jl JSON file

    Returns:
        Dictionary with 'info' and DataFrames for each component type
    """
    # Extract and validate parameters
    info = _extract_parameters(data.get("Parameters", {}))

    # Process each component type
    gen_df, gencost_df = _process_generators(data)
    bus_df, bus_ts_df = _process_buses(data)
    storage_df = _process_storage(data)
    ps_load_df = _process_price_sensitive_loads(data)
    branch_df = _process_transmission_lines(data)
    reserve_df, reserve_ts_df = _process_reserves(data)
    contingency_df = _process_contingencies(data)

    # Create gen_name, bus_name, branch_name for MATPOWER compatibility
    gen_name_df = (
        gen_df[["GEN_NAME"]].copy() if "GEN_NAME" in gen_df.columns else pd.DataFrame()
    )
    bus_name_df = (
        bus_df[["BUS_NAME"]].copy() if "BUS_NAME" in bus_df.columns else pd.DataFrame()
    )
    branch_name_df = (
        branch_df[["BRANCH_NAME"]].copy()
        if "BRANCH_NAME" in branch_df.columns
        else pd.DataFrame()
    )

    return {
        "version": info["version"],
        "baseMVA": info["baseMVA"],
        "bus": bus_df,
        "gen": gen_df,
        "branch": branch_df,
        "gencost": gencost_df,
        "bus_name": bus_name_df,
        "gen_name": gen_name_df,
        "branch_name": branch_name_df,
        "bus_ts": bus_ts_df,
        "reserve": reserve_df,
        "reserve_ts": reserve_ts_df,
        "storage": storage_df,
        "ps_load": ps_load_df,
        "contingency": contingency_df,
        "info": info,
    }


def bus_ts_to_matrix(
    bus_df: pd.DataFrame, bus_ts_df: pd.DataFrame, column: str = "PD"
) -> np.ndarray:
    """
    Convert long-format bus time-series to 2D matrix (buses × time).

    Args:
        bus_df: Bus DataFrame with BUS_I, BUS_NAME columns
        bus_ts_df: Time-series DataFrame with BUS_NAME, TIME, COLUMN, VALUE columns
        column: Column name to extract (default: "PD")

    Returns:
        2D NumPy array with shape (n_buses, n_timesteps)
        Rows correspond to bus_df.BUS_I order

    Example:
        >>> load_matrix = bus_ts_to_matrix(bus_df, bus_ts_df, "PD")
        >>> load_matrix.shape  # (14, 24) for 14 buses, 24 hours
    """
    if bus_ts_df.empty:
        # No time-series data, return static values as single column
        return bus_df[column].values.reshape(-1, 1)

    # Filter to specific column
    column_data = bus_ts_df[bus_ts_df["COLUMN"] == column].copy()

    if column_data.empty:
        # Column not in time-series, return static values
        return bus_df[column].values.reshape(-1, 1)

    # Pivot long format to matrix
    wide_df = column_data.pivot(index="BUS_NAME", columns="TIME", values="VALUE")

    # Reindex to match bus_df order (respecting BUS_I)
    bus_order = bus_df.set_index("BUS_NAME").index
    wide_df = wide_df.reindex(bus_order, fill_value=0.0)

    # Convert to NumPy array
    load_matrix = wide_df.values

    return load_matrix


def reserve_ts_to_matrix(
    reserve_df: pd.DataFrame, reserve_ts_df: pd.DataFrame, column: str = "AMOUNT_MW"
) -> np.ndarray:
    """
    Convert long-format reserve time-series to 2D matrix (reserves × time).

    Args:
        reserve_df: Reserve DataFrame with RESERVE_NAME column
        reserve_ts_df: Time-series DataFrame with RESERVE_NAME, TIME, COLUMN,
        VALUE columns
        column: Column name to extract (default: "AMOUNT_MW")

    Returns:
        2D NumPy array with shape (n_reserves, n_timesteps)
        Rows correspond to reserve_df order
    """
    if reserve_ts_df.empty:
        # No time-series data, return static amounts as single column
        return reserve_df[column].values.reshape(-1, 1)

    # Filter to specific column
    column_data = reserve_ts_df[reserve_ts_df["COLUMN"] == column].copy()

    if column_data.empty:
        # Column not in time-series, return static values
        return reserve_df[column].values.reshape(-1, 1)

    # Pivot long format to matrix
    wide_df = column_data.pivot(index="RESERVE_NAME", columns="TIME", values="VALUE")

    # Reindex to match reserve_df order
    reserve_order = reserve_df["RESERVE_NAME"]
    wide_df = wide_df.reindex(reserve_order, fill_value=0.0)

    # Convert to NumPy array
    reserve_matrix = wide_df.values

    return reserve_matrix


def matrix_to_bus_ts(
    load_matrix: np.ndarray, bus_df: pd.DataFrame, column: str = "PD"
) -> pd.DataFrame:
    """
    Convert 2D matrix back to long-format bus time-series.

    Args:
        load_matrix: 2D array with shape (n_buses, n_timesteps)
        bus_df: Bus DataFrame to get BUS_NAME order
        column: Column name for values (default: "PD")

    Returns:
        Long-format DataFrame with BUS_NAME, TIME, COLUMN, VALUE columns
    """
    n_buses, n_timesteps = load_matrix.shape

    # Create long-format records
    records = []
    for i, bus_name in enumerate(bus_df["BUS_NAME"]):
        for t in range(n_timesteps):
            records.append(
                {
                    "BUS_NAME": bus_name,
                    "TIME": t,
                    "COLUMN": column,
                    "VALUE": load_matrix[i, t],
                }
            )

    return pd.DataFrame(records)

In [8]:
dfs = unitcommitmentjl_to_matpower(data)
for attribute, df in dfs.items():
    display(attribute, df)

'version'

'UnitCommitment.jl-0.3'

'baseMVA'

100.0

'bus'

...

""


'contingency'

,CONTINGENCY_NAME,AFFECTED_GENERATORS,AFFECTED_LINES
0,c1,[],[l1]
1,c2,[],[l2]
2,c3,[],[l3]
3,c4,[],[l4]
4,c5,[],[l5]
5,c6,[],[l6]
6,c7,[],[l7]
7,c8,[],[l8]
8,c9,[],[l9]
9,c10,[],[l10]


'info'

{'version': 'UnitCommitment.jl-0.3',
 'baseMVA': 100.0,
 'time_horizon_min': None,
 'time_horizon_h': 36,
 'time_step_min': 60,
 'power_balance_penalty': 1000.0,
 'scenario_name': 's1',
 'scenario_weight': 1.0}

In [ ]:
# Convert long → matrix for computation
load_matrix = bus_ts_to_matrix(dfs["bus"], dfs["bus_ts"])
reserve_matrix = reserve_ts_to_matrix(dfs["reserve"], dfs["reserve_ts"])

print(f"Load matrix shape: {load_matrix.shape}")  # e.g., (14, 24)
print(f"Reserve matrix shape: {reserve_matrix.shape}")  # e.g., (3, 24)

# Now use in matrix operations (FAST!)
# Example: Multiply by some coefficient matrix
coefficients = np.random.rand(14, 5)  # (buses, features)
result = coefficients.T @ load_matrix  # (5, 24) - FAST dense multiplication

# Example that preserves bus dimension (works with matrix_to_bus_ts)
time_scaling = np.ones(36)  # Scale each time period
scaled_loads = load_matrix * time_scaling  # (14, 36) ← still 14 buses!
bus_ts_scaled = matrix_to_bus_ts(scaled_loads, dfs["bus"])  # ✅ Works!

# Example that changes dimensions (needs different handling)
coefficients = np.random.rand(14, 5)
result = coefficients.T @ load_matrix  # (5, 36) ← only 5 features!

Load matrix shape: (14, 36)
Reserve matrix shape: (1, 36)
